### "General normalizing flow set-up:"

> I think a "general normalizing flow" should be introduced in it's own section rather than within and IAF heading so that it's not confused/conflated with IAF specifically.

$\mathbf{z}_0 \sim q(\mathbf{z}_0| \mathbf{x}), \mathbf{z}_t = f_t(\mathbf{z}_{t-1}, \mathbf{x})$

> The role and notation of x in the first equation is not clear to me.

### "Motivation of IAF:"
> I suspect planar/radial is not useful in high dimensions both from a computational perspective as well as in terms of its representational capability (in a similar way that SVMs cannot generalize as well as NNs in some settings), but I am not sure what you're intending to indicate with your statements in this section.

For $i>0$, $z^t_i = \boldsymbol \mu^t_{i}(\mathbf{z}^{t-1}_{0:i-1}) + \boldsymbol \sigma^t_{i}(\mathbf{z}^{t-1}_{0:i-1})*z^{t-1}_i$
> $\mu^t_{i}$ and $\sigma^t_{i}$ should be bolded


a single forward pass $\mathbf{y} = f_T\circ \dots \circ f_1(\mathbf{z})$ can be computed parallelly and efficiently, which benefits the sampling process.

> For a given $t$, the parallelization is across $i$ in your $z_i^t$ notation; but, processing across $t$ must still be sequential. So, the statement that the composition "can be computed parallelly and efficiently" could more precisely be "at each stage of the composition the univariate elements of transformation $f_t$ can thus be computed parallelly and efficiently"

> I think the efficiency considerations could more readily emerge if the writing was more oriented around illustrating this aspect of IAF.  From this it can become clear where IAF attempts to scale better than e.g., planar/radial flows: it is across the elements of $z_t$ which we may parallelize under this methodology, but the composition itself must still proceed sequentially.

> The remainder of this section was must better in this regard; though, $\mathbf{z}_{t-1}$ does not "depend on its own (first $i - 1$) elements", $z^{t-1}_i$ does (and so we may not parallelize the elements of z within each inverse transformation of the composition)

### "Masked Autoregressive Flow (MAF)"

> I think that compared to the general $z_t = f_t(z_{t-1})$ notation, the specific notations 
- $z_t = f_{z_{t-1}}(z_{t-1})$ for IAF 
- $z_t = g_{z_t}(z_{t-1})$ for MAF 
>
> are more useful since they show that 
- IAF forward elements $z_i^t$, given $z_{t-1}$, can be computed in parallel
- whereas for MAF we can see that it is the elements $z_i^{t-1}$ of the inverse $z_{t-1} = g_{z_t}^{-1}(z_t)$ which can be computed in parallel given $z_{t}$.

> The IAF inverse $z_{t-1} = f_{z_{t-1}}^{-1}(z_t)$ is the "inverse" of the MAF forward  $z_t = g_{z_t}(z_{t-1})$ in the sense that the positions of $z_t$ and $z_{t-1}$ are switched, and the same is true of the IAF forward relative to the MAF inverse. 
> - These role reversals, and the IAF moniker, follow directly from
>  
>   `bijector=tfb.Invert(tfb.MaskedAutoregressiveFlow(made)))`
>
>   which 
>   1. does not change `shift_and_log_scale_fn=made`
>   2. but instead just swaps $f$ for $g^{-1}$ and $f^{-1}$ for $g$ so 
>
>      IAF forward is now $z_t = f_{z_{t-1}}(z_{t-1}) = g^{-1}_{z_{t-1}}(z_{t-1})$ `# rather than MAF usage` $g_{z_t}^{-1}(z_t)$
>       
>      IAF inverse is now $z_{t-1} = f_{z_{t-1}}^{-1}(z_t) = g_{z_{t-1}}(z_t)$ `# rather than MAF usage` $g_{z_t}(z_{t-1})$
>      <br>
>
>   3. which again notationally indicates the parallelizablity of the IAF foward pass (previously the MAF inverse pass)
       - which is also seen in swapping the `forward` and `inverse` computations below:
       

In [2]:
# from
# https://www.tensorflow.org/probability/api_docs/python/tfp/bijectors/MaskedAutoregressiveFlow

def forward(x):
  y = zeros_like(x)
  event_size = x.shape[-event_dims:].num_elements()
  for _ in range(event_size):
    shift, log_scale = shift_and_log_scale_fn(y)
    y = x * tf.exp(log_scale) + shift
  return y

def inverse(y):
  shift, log_scale = shift_and_log_scale_fn(y)
  return (y - shift) / tf.exp(log_scale)


Both IAF and MAF use MADE as their component layer. MAF is a normalizing flow that represents a invertable transformation applied to a distribution. MADE can be considered as a (network) layer, while MAF is a bijector with inverse operation. 
> IAF also is a bijector, and bijectors also have log det Jacobians for forward and inverse transformations.
> 
> I think it's also worth noting that MADE layer outputs can define autoregressive probability distributions defining a joint distribution, e.g., of the form
>
> $$\mathbf{z} \sim N(\boldsymbol \mu(\mathbf{z}), \text{diag}(\boldsymbol \sigma(\mathbf{z}))$$
>
> which is equivalent to the single bijection IAF defined in your "estimate density using IAF" section but
> 1. can be sampled from directly
> 2. can be evaluated directly

# TFP implementation using MAF: 1) 2D example

- You need `input_order='right-to-left'` so that you first model x2 and then x1|x2
    - x2|x1 in the moon example is bimodal so not a nice natural way to condition
- But then you also need a nonlinear transform so that 
    - x1 increases as x2<0 decreasing
    - but also x1 increases for x2>0 increasing
         - 'relu' is a usual choice for that
    - and you need a good number of hidden units which combine their relu's to make the nonlinarity smooth
        - e.g., `hidden_units=[50,50]` 
        
Not sure they did any of this very well in the [TF docs](https://www.tensorflow.org/probability/api_docs/python/tfp/bijectors/MaskedAutoregressiveFlow)...

#### *But I like that you almost got your code working nonetheless!*
- This below will do what you want:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_probability as tfp
tfb = tfp.bijectors
tfd = tfp.distributions
tfk = tf.keras
tfkl = tf.keras.layers

# Generate data X1 = X2^2 / 4

n = 2000
x2 = np.random.randn(n).astype(dtype=np.float32) * 2.
x1 = np.random.randn(n).astype(dtype=np.float32) + (x2 * x2 / 4.)
data = np.stack([x1, x2], axis=-1)
plt.plot(x1, x2, ".")
plt.xlabel("X1")
plt.ylabel("X2")
plt.show()

In [ ]:
# A layer function of MADE (bijector function for MAF)
made1 = tfb.AutoregressiveNetwork(params=2, hidden_units=[50,50], 
                                  activation='relu', input_order='right-to-left') # <- THIS

# Target distrbution transformed by MAF (3 layers)
distribution = tfd.TransformedDistribution(
    distribution=tfd.MultivariateNormalDiag(loc=[0,0], scale_diag=[1,1]), # base: 2D Gaussian
    bijector=tfb.MaskedAutoregressiveFlow(made1))
# distribution = tfd.TransformedDistribution(distribution, bijector=tfb.MaskedAutoregressiveFlow(made2))
# distribution = tfd.TransformedDistribution(distribution, bijector=tfb.MaskedAutoregressiveFlow(made3))

# fit tf model
inputs = tfkl.Input(shape=(2,))
outputs = distribution.log_prob(inputs)
model = tfk.Model(inputs, outputs)

def neg_log(true, output):
  return -output
model.compile(optimizer=tf.optimizers.Adam(), loss=neg_log)
model.fit(x=data, y=np.zeros(n), epochs=50, verbose=False) # y does not matter (unsupervised)

# Sampling from the estimated density for visualization

samples = distribution.sample(1000)
x1_hat = samples[:, 0]
x2_hat = samples[:, 1]
plt.plot(x1_hat, x2_hat, "r.")
plt.xlabel("X1")
plt.ylabel("X2")

g = 100
g1,g2 = np.meshgrid(np.linspace(x1_hat.numpy().min(), x1_hat.numpy().max(), g), 
                  np.linspace(x2_hat.numpy().min(), x2_hat.numpy().max(), g))
grid = np.stack([g1.flatten(), g2.flatten()], axis=-1)
plt.contour(g1, g2, distribution.prob(grid).numpy().reshape(g,g), 
            levels=np.logspace(-5,2,10))
plt.show()

In [ ]:
# MADE layer
made = tfb.AutoregressiveNetwork(params=2, hidden_units=[512,512],
                                 activation='relu', input_order='right-to-left') # <- THIS
# IAF model 
iaf = istribution = tfd.TransformedDistribution(
    distribution=tfd.MultivariateNormalDiag(loc=[0,0], scale_diag=[1,1]), # base: 2D Gaussian
    bijector=tfb.Invert(tfb.MaskedAutoregressiveFlow(made)))  # just need to invert the maf
# fit model
inputs = tfkl.Input(shape=(2,))
outputs = distribution.log_prob(inputs)
model = tfk.Model(inputs, outputs)

model.compile(optimizer=tf.optimizers.Adam(), loss=neg_log)
model.fit(x=data, y=np.zeros(n), epochs=50, verbose=False)

In [ ]:
# Sampling from the estimated density for visualization

samples = distribution.sample(1000)
x1_hat = samples[:, 0]
x2_hat = samples[:, 1]
plt.plot(x1_hat, x2_hat, "r.")
plt.xlabel("X1")
plt.ylabel("X2")
g = 100
g1,g2 = np.meshgrid(np.linspace(x1_hat.numpy().min(), x1_hat.numpy().max(), g), 
                  np.linspace(x2_hat.numpy().min(), x2_hat.numpy().max(), g))
grid = np.stack([g1.flatten(), g2.flatten()], axis=-1)
plt.contour(g1, g2, distribution.prob(grid).numpy().reshape(g,g), 
            levels=np.logspace(-5,2,10))
plt.show()